In [1]:
%pip install contractions
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import contractions

import nltk
nltk.download('wordnet', quiet=True)
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
data_path = "/home/jupyter/src/natalies-little-helper/data/"
raw_data = pd.read_csv(data_path + "Tweets.csv")

raw_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [17]:
def clean_data(x):
    x = x.lower() # convert to all lower case
    # x = re.sub('http\S+', '', x) # remove URLs
    # x = re.sub('<.*?>', '', x) # remove HTML tags
    x = contractions.fix(x) # expand contractions
    x = re.sub('[^a-z]', ' ', x) # remove non-alphabetic characters
    x = re.sub('\s\s+', ' ', x) # remove extra whitespace
    return x

In [18]:
df = raw_data.copy(deep=True)
df.text = df.text.apply(clean_data)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica plus you have added commercials...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,virginamerica i did not today must mean i nee...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,virginamerica it is really aggressive to blas...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica and it is a really big bad thin...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [19]:
stopwords_eng = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

remove_stopwords = lambda x: ' '.join([word for word in word_tokenize(x) if word not in stopwords_eng])
lemmatizer = lambda x: ' '.join(
    [
        wnl.lemmatize(
            wnl.lemmatize(
                wnl.lemmatize(
                    wnl.lemmatize(
                        word,
                        pos='n'
                    ),
                pos='v'
                ),
            pos='a'
            ),
        pos='r'
        ) 
        for word in word_tokenize(x)
    ]
)
# def preprocess(x):
#     x = remove_stopwords(x)
#     x = lemmatizer(x)
#     return x

# df.text = df.text.apply(preprocess)
# df.head()

In [20]:
def convert_labels(x):
    x = x.strip()
    if x == "positive":
        return 1
    elif x == "negative":
        return -1
    elif x == "neutral":
        return 0
df.airline_sentiment = df.airline_sentiment.apply(convert_labels)

In [21]:
df.to_csv(data_path + "cleaned_Tweets.csv")

## Intent Tweets Pre-processing

In [22]:
intent = df[df['negativereason'].notna()]
intent.reset_index(drop=True, inplace=True)

In [23]:
# Customer Service Issue         2910
# Late Flight                    1665
# Can't Tell                     1190
# Cancelled Flight                847
# Lost Luggage                    724
# Bad Flight                      580
# Flight Booking Problems         529
# Flight Attendant Complaints     481
# longlines                       178
# Damaged Luggage                  74

intentSubGroups = {
    'Customer Service Issue': 'Customer Service',
    'Late Flight': 'Flights',
    'Cancelled Flight': 'Flights',
    'Lost Luggage': 'Luggage',
    'Bad Flight': 'Flights',
    'Flight Booking Problems': 'Customer Service',
    'Flight Attendant Complaints': 'Customer Service',
    'longlines': 'Customer Service',
    'Damaged Luggage': 'Luggage',
    'Can\'t Tell': 'Can\'t Tell'
}

intent['intentGroup'] = intent.negativereason.apply(lambda x: intentSubGroups[x])
intent

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,intentGroup
0,570301031407624196,-1,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,virginamerica it is really aggressive to blas...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),Flights
1,570300817074462722,-1,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica and it is a really big bad thin...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),Can't Tell
2,570300767074181121,-1,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,virginamerica seriously would pay a flight fo...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada),Can't Tell
3,570282469121007616,-1,0.6842,Late Flight,0.3684,Virgin America,NaN,smartwatermelon,NaN,0,virginamerica sfo pdx schedule is still mia,NaN,2015-02-24 10:01:50 -0800,"palo alto, ca",Pacific Time (US & Canada),Flights
4,570276917301137409,-1,1.0000,Bad Flight,1.0000,Virgin America,NaN,heatherovieda,NaN,0,virginamerica i flew from nyc to sfo last wee...,NaN,2015-02-24 09:39:46 -0800,this place called NYC,Eastern Time (US & Canada),Flights
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9173,569588464896876545,-1,1.0000,Bad Flight,1.0000,American,NaN,MDDavis7,NaN,0,americanair thanks for nothing on getting us ...,NaN,2015-02-22 12:04:07 -0800,US,Eastern Time (US & Canada),Flights
9174,569587705937600512,-1,1.0000,Cancelled Flight,1.0000,American,NaN,RussellsWriting,NaN,0,americanair my flight was cancelled flightled...,NaN,2015-02-22 12:01:06 -0800,Los Angeles,Arizona,Flights
9175,569587691626622976,-1,0.6684,Late Flight,0.6684,American,NaN,GolfWithWoody,NaN,0,americanair right on cue with the delays,NaN,2015-02-22 12:01:02 -0800,NaN,Quito,Flights
9176,569587371693355008,-1,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,americanair leaving over minutes late flight ...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,Customer Service


In [24]:
intent.to_csv(data_path + "intent_Tweets.csv")